In [2]:
import pandas as pd

In [4]:
all_sui_reg = pd.read_csv('Data/all_suicides/suicidios-reg.csv')
print(all_sui_reg)

    cod;nome;período;valor
0           3;SE;1989;1971
1             1;N;1989;220
2            5;CO;1989;349
3            2;NE;1989;540
4            4;S;1989;1411
..                     ...
150         2;NE;2019;3082
151         5;CO;2019;1283
152         3;SE;2019;4930
153          1;N;2019;1058
154          4;S;2019;3167

[155 rows x 1 columns]


In [5]:
# All of the files are csv, with separators ';'
# Gonna try to adjust our Df accordingly

all_sui_reg = pd.read_csv('Data/all_suicides/suicidios-reg.csv',sep=';')
print(all_sui_reg)

     cod nome  período  valor
0      3   SE     1989   1971
1      1    N     1989    220
2      5   CO     1989    349
3      2   NE     1989    540
4      4    S     1989   1411
..   ...  ...      ...    ...
150    2   NE     2019   3082
151    5   CO     2019   1283
152    3   SE     2019   4930
153    1    N     2019   1058
154    4    S     2019   3167

[155 rows x 4 columns]


In [7]:
# We need to adjust every df to plot them later on, and to have easier access to the data
# For now we are going to adjust the regions, states(UFs), Men and Womens data.

all_sui_uf = pd.read_csv('Data/all_suicides/suicidios-uf.csv',sep=';')
men_sui_reg = pd.read_csv('Data/men/suicidios-de-homens-reg.csv',sep=';')
wom_sui_reg = pd.read_csv('Data/women/suicidio-de-mulheres-reg.csv',sep=';')

In [8]:
# Let's take a look at how we can filter data
print(wom_sui_reg)

    cod nome  período  valor
0     5   CO     2000    157
1     1    N     2000     84
2     2   NE     2000    261
3     3   SE     2000    499
4     4    S     2000    381
..  ...  ...      ...    ...
95    1    N     2019    231
96    4    S     2019    653
97    3   SE     2019   1117
98    2   NE     2019    616
99    5   CO     2019    302

[100 rows x 4 columns]
